# RAG Practical Usecase
1.Document Question Answering Systems
2.Conversational agents
3.Real-time Event Commentary
4.Content Generation
5.Personalised Recommendation
6.Virtual Assistants

In [3]:
from dotenv import load_dotenv
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)



## part one Data Ingestion

In [6]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [15]:
loader = TextLoader('state_of_the_union.txt', encoding="utf8")

In [24]:
document = loader.load()
# print(document[0].page_content)

In [26]:
# chunking the data
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
text_chunks = text_splitter.split_documents(document)
text_chunks[0].page_content



'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.'

In [56]:
# from langchain.embeddings import OpenAIEmbeddings                   
from langchain_community.embeddings import HuggingFaceEmbeddings  ## this is free if you want to you any free embedder
from langchain.vectorstores import FAISS

In [57]:
# embeddings = OpenAIEmbeddings(openai_api_type=OPENAI_API_KEY)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_52515/1450039401.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/no0ne/Documents/machine-learning/Generative-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

## Data retrieval 

In [ ]:
retrieverr  = vectorstore.as_retriever()

In [60]:
from langchain.prompts import ChatPromptTemplate

In [61]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [62]:
prompt = ChatPromptTemplate.from_template(template)

In [63]:
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [64]:
output_parser = StrOutputParser()

In [65]:
llm_model = ChatOpenAI(model="gpt-3.5-turbo")

In [66]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

NameError: name 'retriever' is not defined

In [ ]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")